# Segmenting and Clustering Neighborhoods in Toronto

## Load Libraries

In [119]:
# Import Potential Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

!pip install folium==0.5.0
import folium





## Load and Prepare Data

In [120]:
# load postcodes to data from wiki

dfpct = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [121]:
# create datadframe

dfpc = dfpct[0]

# check dataframe

dfpc.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [122]:
# drop rows where borough unassigned

dfpc = dfpc[dfpc.Borough != 'Not assigned']

#check dataframe

dfpc.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [123]:
# check for any Neighbourhoods that are same as Borough

print (dfpc[dfpc['Borough']==dfpc['Neighbourhood']])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [124]:
# check each postal code has a unique record

print ('number of unique postcodes',dfpc['Postal Code'].nunique())

print ('number of rows',dfpc['Postal Code'].count())


number of unique postcodes 103
number of rows 103


The postcode file has been loaded into the dataframe pfpc and any unassigned boroughs dropped.  It has also been checked to see if any Neighbourhoods contain the name of the Borough.  None do.  Where postcode contains two neighbourhoods these are combined and seperated by a comma.  As the number of unique postcodes is the same as the numver of rows each postcode is unique.

In [125]:
# print number of rows and columns

print ('number of rows and columns')
dfpc.shape

number of rows and columns


(103, 3)

 ## Add Latitude and Longitude

In [126]:
# Load Latitude and Longitude File

dfll=pd.read_csv('https://cocl.us/Geospatial_data')

In [127]:
# check dataframe

dfll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [128]:
# Check number of postal codes

print ('number of rows and columns')
dfll.shape

number of rows and columns


(103, 3)

Number of rows in postcode and location files are the same

In [129]:
# Join locations to postcode file using 'Postal Code'

dfm=pd.merge(dfpc, dfll, how='inner', on='Postal Code')

In [130]:
# Check joined dataframe

dfm.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [131]:
# Check value of Postal Code 'M5G'

dfc=dfm.loc[dfm['Postal Code']=='M5G']

print (dfc)

   Postal Code           Borough       Neighbourhood   Latitude  Longitude
24         M5G  Downtown Toronto  Central Bay Street  43.657952 -79.387383


This row cheks with course example

## Explore Data

Create a map of Toronto

In [132]:
## Get latitude and longitude for Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [133]:
# create map of Totonto using latitude and longitude values
map_t = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfm['Latitude'], dfm['Longitude'], dfm['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)  
    
map_t


## Load FourSquare data

In [134]:
# Define Four Square credentials

CLIENT_ID = 'WIP2W2FQF0TEYU3TROAJTQVNCHFYO4OTJWH1ZJWLBAI24FYL' # your Foursquare ID
CLIENT_SECRET = 'QDCKSGFJT1P3WF3J4HTIQCC4VN2W54XTJQ5ALW5EOGH05UWQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
# Create list of all venues in Borough

toronto_venues = getNearbyVenues(names=dfm['Borough'],
                                   latitudes=dfm['Latitude'],
                                   longitudes=dfm['Longitude']
                                  )

# Check Data Frame

print(toronto_venues.shape)
toronto_venues.head()


North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


## Check the 

In [137]:
# Check number of venues
toronto_venues.shape

(2141, 7)

In [138]:
# Create list of how many venues are in each Neighbourhood

toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,104,104,104,104,104,104
Downtown Toronto,1248,1248,1248,1248,1248,1248
East Toronto,119,119,119,119,119,119
East York,79,79,79,79,79,79
Etobicoke,74,74,74,74,74,74
Mississauga,13,13,13,13,13,13
North York,241,241,241,241,241,241
Scarborough,90,90,90,90,90,90
West Toronto,153,153,153,153,153,153


In [139]:
# List of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [140]:
# List of what is in each Borough

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Prepare the data to allow the clustering by looking at the ferequency of each venue

In [141]:
# Create frequency DataFrame to enable Clustering

toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [142]:
# Check Shape

toronto_grouped.shape

(10, 274)

## Find the top 10 to make it easier to analyse the clusters

In [143]:
# Function to return most popular venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
     venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Bagel Shop,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Restaurant,Park,Ice Cream Shop,Pizza Place,American Restaurant,Bakery
3,East York,Coffee Shop,Bank,Intersection,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Pharmacy,Indian Restaurant
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery


## Cluster the areas

In [145]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1], dtype=int32)

In [110]:
# add clustering labelstoronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
# Add latitude and longitude back in

venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged=dfm

#toronto_merged=toronto_merged.join(toronto_grouped.setindex('Borough'), on'Borough')

#toronto_merged.head()

toronto_merged=pd.merge(toronto_merged, venues_sorted, how='inner', on='Borough')

toronto_merged.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,Parkwoods,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
1,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"Lawrence Manor, Lawrence Heights",2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
3,M3B,North York,Don Mills,43.745906,-79.352188,Don Mills,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
4,M6B,North York,Glencairn,43.709577,-79.445073,Glencairn,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café


## Create the map

In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## A breakdown of the top 10 for ecach cluster

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Scarborough,"Malvern, Rouge",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
56,Scarborough,"Rouge Hill, Port Union, Highland Creek",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
57,Scarborough,"Guildwood, Morningside, West Hill",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
58,Scarborough,Woburn,0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
59,Scarborough,Cedarbrae,0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
60,Scarborough,Scarborough Village,0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
61,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
62,Scarborough,"Golden Mile, Clairlea, Oakridge",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
63,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
64,Scarborough,"Birch Cliff, Cliffside West",0,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant


In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,York,Humewood-Cedarvale,1,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field
78,York,Caledonia-Fairbanks,1,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field
79,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",1,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field
80,York,"Runnymede, The Junction North",1,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field
81,York,Weston,1,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field


In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
1,North York,Victoria Village,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
2,North York,"Lawrence Manor, Lawrence Heights",2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
3,North York,Don Mills,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
4,North York,Glencairn,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
5,North York,Don Mills,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
6,North York,Hillcrest Village,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
7,North York,"Bathurst Manor, Wilson Heights, Downsview North",2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
8,North York,"Fairview, Henry Farm, Oriole",2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
9,North York,"Northwood Park, York University",2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café


In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Mississauga,Canada Post Gateway Processing Centre,3,Coffee Shop,Hotel,Middle Eastern Restaurant,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Gas Station,American Restaurant


In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood2,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Etobicoke,"Islington Avenue, Humber Valley Village",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
44,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
45,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
46,Etobicoke,Westmount,4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
47,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
48,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
49,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
50,Etobicoke,"Alderwood, Long Branch",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
51,Etobicoke,"Northwest, West Humber - Clairville",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
52,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",4,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery


# About the Clusters

## Cluster 0: The Meeting Point

Great for grabbing something from the bakery or meeting up for ethnic cuisine

## Cluster 1: Out and About

Hop on the bus and take some exercise in the park or on the tennis court

## Cluster 2: If you are Visiting

The place to be if you are looking for a Hotel, Restraunt or perhaps a enliveing coffee

## Cluster 3: Too Many

Given there is only one place in this cluster 4 might be a better number of clusters than 5

## Cluster 4: Grab a Bite

If you ae looking for a sandwich or pizza this is the place to be



